# Torre de Hanoi - Búsqueda en Profundidad Iterativa

Este notebook resuelve el problema de la Torre de Hanoi utilizando el algoritmo de Búsqueda en Profundidad Iterativa (IDDFS).

In [16]:
## Setup
import time
import statistics
import psutil
import os

## 1. Introducción

El problema de la Torre de Hanoi consiste en mover una pila de discos de una clavija a otra, siguiendo estas reglas:
1. Solo se puede mover un disco a la vez.
2. Un disco más grande no puede colocarse sobre uno más pequeño.
3. Solo se puede mover el disco superior de una pila.

La Búsqueda en Profundidad Iterativa (IDDFS) es un algoritmo de búsqueda que combina la eficiencia de espacio de la búsqueda en profundidad con la completitud de la búsqueda en anchura.

In [3]:
# Implementación del algoritmo IDDFS para la Torre de Hanoi

class Estado:
    def __init__(self, pegs):
        self.pegs = pegs

    def __eq__(self, other):
        return self.pegs == other.pegs

    def __hash__(self):
        return hash(tuple(tuple(peg) for peg in self.pegs))

    def __str__(self):
        return str(self.pegs)

def mover_disco(estado, origen, destino):
    nuevo_estado = [list(peg) for peg in estado.pegs]
    disco = nuevo_estado[origen].pop()
    nuevo_estado[destino].append(disco)
    return Estado(nuevo_estado)

def es_movimiento_valido(estado, origen, destino):
    if not estado.pegs[origen]:
        return False
    if estado.pegs[destino] and estado.pegs[origen][-1] > estado.pegs[destino][-1]:
        return False
    return True

def obtener_sucesores(estado):
    sucesores = []
    for origen in range(3):
        for destino in range(3):
            if origen != destino and es_movimiento_valido(estado, origen, destino):
                sucesores.append(mover_disco(estado, origen, destino))
    return sucesores

def es_objetivo(estado, num_discos):
    return len(estado.pegs[2]) == num_discos

def iddfs(estado_inicial, num_discos):
    for limite_profundidad in range(1, 2 ** num_discos):
        resultado = dfs_limitado(estado_inicial, num_discos, limite_profundidad)
        if resultado is not None:
            return resultado
    return None

def dfs_limitado(estado, num_discos, limite_profundidad, profundidad=0, visitados=None):
    if visitados is None:
        visitados = set()

    if profundidad > limite_profundidad:
        return None

    if es_objetivo(estado, num_discos):
        return []

    visitados.add(estado)

    for sucesor in obtener_sucesores(estado):
        if sucesor not in visitados:
            resultado = dfs_limitado(sucesor, num_discos, limite_profundidad, profundidad + 1, visitados)
            if resultado is not None:
                return [(estado, sucesor)] + resultado

    visitados.remove(estado)
    return None

def resolver_torre_hanoi(num_discos):
    estado_inicial = Estado([[i for i in range(num_discos, 0, -1)], [], []])
    solucion = iddfs(estado_inicial, num_discos)
    return solucion

# Ejemplo de uso
num_discos = 3
solucion = resolver_torre_hanoi(num_discos)

if solucion:
    print(f"Solución encontrada en {len(solucion)} movimientos:")
    for i, (estado, siguiente_estado) in enumerate(solucion, 1):
        print(f"Movimiento {i}: {estado} -> {siguiente_estado}")
else:
    print("No se encontró solución.")

Solución encontrada en 7 movimientos:
Movimiento 1: [[3, 2, 1], [], []] -> [[3, 2], [], [1]]
Movimiento 2: [[3, 2], [], [1]] -> [[3], [2], [1]]
Movimiento 3: [[3], [2], [1]] -> [[3], [2, 1], []]
Movimiento 4: [[3], [2, 1], []] -> [[], [2, 1], [3]]
Movimiento 5: [[], [2, 1], [3]] -> [[1], [2], [3]]
Movimiento 6: [[1], [2], [3]] -> [[1], [], [3, 2]]
Movimiento 7: [[1], [], [3, 2]] -> [[], [], [3, 2, 1]]


## 2. PEAS del problema

- **Rendimiento (Performance)**: Minimizar el número de movimientos para resolver el problema.
- **Entorno (Environment)**: Las tres clavijas y los discos.
- **Actuadores (Actuators)**: Mover discos entre clavijas.
- **Sensores (Sensors)**: Observar el estado actual de las clavijas y los discos.

## 3. Propiedades del entorno de trabajo

- Completamente observable
- Determinista
- Secuencial
- Estático
- Discreto
- Agente único

## 4. Elementos del problema

- **Estado**: Configuración actual de los discos en las clavijas
- **Espacio de estados**: Todas las posibles configuraciones de discos en las clavijas
- **Árbol de búsqueda**: Árbol que representa todos los movimientos posibles y los estados resultantes
- **Nodo de búsqueda**: Un nodo en el árbol de búsqueda, que representa un estado y el movimiento que llevó a él
- **Objetivo**: Todos los discos en la clavija de destino en el orden correcto
- **Acción**: Mover un disco de una clavija a otra
- **Frontera**: Conjunto de nodos no expandidos en el árbol de búsqueda

## 5. Complejidad del algoritmo

- **Complejidad temporal**: O(b^d), donde b es el factor de ramificación y d es la profundidad
- **Complejidad espacial**: O(d)

## 6. Análisis de rendimiento empírico

Ejecutaremos el algoritmo 10 veces para diferentes números de discos y mediremos el tiempo de ejecución y el uso de memoria.

In [5]:
def medir_rendimiento(num_discos, num_ejecuciones=2):
    tiempos = []
    memorias = []
    
    for _ in range(num_ejecuciones):
        inicio = time.time()
        proceso = psutil.Process(os.getpid())
        mem_inicio = proceso.memory_info().rss
        
        resolver_torre_hanoi(num_discos)
        
        fin = time.time()
        mem_fin = proceso.memory_info().rss
        
        tiempos.append(fin - inicio)
        memorias.append(mem_fin - mem_inicio)
    
    return {
        'tiempo_promedio': statistics.mean(tiempos),
        'tiempo_desviacion': statistics.stdev(tiempos),
        'memoria_promedio': statistics.mean(memorias),
        'memoria_desviacion': statistics.stdev(memorias)
    }

for discos in range(3, 8):
    resultados = medir_rendimiento(discos)
    print(f"Discos: {discos}")
    print(f"Tiempo promedio: {resultados['tiempo_promedio']:.4f} s ± {resultados['tiempo_desviacion']:.4f} s")
    print(f"Memoria promedio: {resultados['memoria_promedio'] / 1024:.2f} KB ± {resultados['memoria_desviacion'] / 1024:.2f} KB")
    print()

Discos: 3
Tiempo promedio: 0.0028 s ± 0.0006 s
Memoria promedio: 28.00 KB ± 33.94 KB

Discos: 4
Tiempo promedio: 0.0888 s ± 0.0053 s
Memoria promedio: 118.00 KB ± 161.22 KB



KeyboardInterrupt: 

## 7. Comparación con la solución óptima

Compararemos el número de movimientos encontrados por IDDFS con la solución óptima (2^k - 1, donde k es el número de discos).

In [ ]:
def comparar_con_optimo(num_discos, num_ejecuciones=10):
    movimientos = []
    for _ in range(num_ejecuciones):
        solucion = resolver_torre_hanoi(num_discos)
        movimientos.append(len(solucion))
    
    promedio_movimientos = statistics.mean(movimientos)
    optimo = 2**num_discos - 1
    diferencia_porcentual = ((promedio_movimientos - optimo) / optimo) * 100
    
    return {
        'promedio_movimientos': promedio_movimientos,
        'optimo': optimo,
        'diferencia_porcentual': diferencia_porcentual
    }

for discos in range(3, 8):
    resultados = comparar_con_optimo(discos)
    print(f"Discos: {discos}")
    print(f"Movimientos promedio: {resultados['promedio_movimientos']:.2f}")
    print(f"Solución óptima: {resultados['optimo']}")
    print(f"Diferencia porcentual: {resultados['diferencia_porcentual']:.2f}%")
    print()

## 8. Conclusión

En este notebook, hemos implementado y analizado el algoritmo de Búsqueda en Profundidad Iterativa (IDDFS) para resolver el problema de la Torre de Hanoi. Hemos examinado sus propiedades, rendimiento y lo hemos comparado con la solución óptima.

El IDDFS demuestra ser efectivo para resolver el problema de la Torre de Hanoi, encontrando soluciones óptimas en la mayoría de los casos. Sin embargo, su rendimiento puede degradarse rápidamente a medida que aumenta el número de discos debido a su naturaleza de búsqueda exhaustiva.

Para problemas con un número pequeño de discos, el IDDFS es una opción viable, pero para problemas más grandes, podrían ser necesarios enfoques más eficientes o heurísticos.